# Project ClearChoise Textile Classifier


#### The aim of this project is to develop a pattern classifier. By inputting an image, the program will provide feedback on the pattern present in the photographed garment. Making mornings easier and more fun for those with visual impairments, helping them pick out their perfect outfit effortlessly!

- First model looks for flowers. flower_model_28-02-24.pt
- Then an model for geometric patterns. geometric_24-03-04.h5

In [23]:
import cv2
import numpy as np
import torch
import tensorflow as tf
import matplotlib.pyplot as plt
torch.__version__

'2.2.1+cpu'

If there is problem with yolo locally:

In [8]:
import pathlib
from pathlib import Path
pathlib.PosixPath = pathlib.WindowsPath

## Load Flowermodel

In [16]:
yolo_path = "C:/Users/emmel/Desktop/yolov5"
model_path = "./Flower_model/flower_model_28-02-24.pt"

yolo_model = torch.hub.load(yolo_path, 'custom', path=model_path, source='local')
yolo_model.conf = 0.30

requirements: protobuf<=3.20.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per C:\Users\emmel\Desktop\yolov5\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  v6.1-362-g731a2f8c Python-3.10.6 torch-2.2.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


## Load Geometricmodel

In [25]:
geometric_model_path = "./geometric_model/geometric_24-03-04.h5" 

geometric_model = tf.keras.models.load_model(geometric_model_path)

In [29]:
img_dim = (64,64)

class_labels = ["check", 
                "stripe", 
                "dots"
]

def load_and_preprocess_image(image_path, img_dim):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, img_dim)
    img = img[:, :, np.newaxis]
    img = img / 255
    return np.expand_dims(img, axis=0)

### Test Images

In [34]:
img_paths = [ "./img_for_test/1.jpg",
                "./img_for_test/8.jpg",
                "./img_for_test/6.jpg",
                ]

In [36]:
for img_path in img_paths:
    img = cv2.imread(img_path)
    yolo_result = yolo_model(img)
    yolo_preds = yolo_result.pandas().xyxy[0]
    
    if len(yolo_preds) > 0:
        #print(f"{img_path}\nFlowers found: {len(yolo_preds)} \n")
        #yolo_result.show()
        print(f"{img_path}\nPredicted pattern: Flowers")
    else:
        img_preprocessed = load_and_preprocess_image(img_path, img_dim)
        prediction = geometric_model.predict(img_preprocessed)
        predicted_class = np.argmax(prediction)
        predicted_probability = prediction[0, predicted_class]

        predicted_label = class_labels[predicted_class]

        print(f"{img_path}\nPredicted pattern: {predicted_label}")
        #print("Predicted Probability: {:.2f}%".format(predicted_probability * 100))

./img_for_test/1.jpg
Predicted pattern: Flowers
1/1 [==============================] - 0s 24ms/step
./img_for_test/8.jpg
Predicted pattern: check
./img_for_test/6.jpg
Predicted pattern: Flowers
